In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# Install these packages with these specific versions else the notebook breaks
!pip install transformers==4.5.1
!pip install pytorch_lightning==1.2.10
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 6.4MB/s 
     |████████████████████████████████| 901kB 22.2MB/s 
     |████████████████████████████████| 3.3MB 39.2MB/s 
     |████████████████████████████████| 849kB 6.7MB/s 
     |████████████████████████████████| 829kB 10.5MB/s 
     |████████████████████████████████| 112kB 22.1MB/s 
     |████████████████████████████████| 276kB 16.7MB/s 
     |████████████████████████████████| 184kB 21.9MB/s 
     |████████████████████████████████| 1.3MB 18.7MB/s 
     |████████████████████████████████| 143kB 38.1MB/s 
     |████████████████████████████████| 296kB 34.8MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=4c7fd9ff9a2cf35e63e60e2b2339bf08a1cb5aaae454c35315ce1714fbc93bc1
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=224ba216e2f5e5b2da16951ea2acf5

In [ ]:
# Import packages
import argparse
import glob
import pickle
import os
import json
import time
import logging
import random
import re
from tqdm import tqdm
from itertools import chain
from string import punctuation
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import sentencepiece


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import csv
from dataclasses import dataclass

from enum import Enum
from typing import List, Optional
from transformers import PreTrainedTokenizer

In [ ]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "val_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [ ]:
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    
    if type(hparams) is dict: 
      hparams = argparse.Namespace(**hparams)
    
    self.hparams = hparams
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)
  
  def is_logger(self):
    return self.trainer.global_rank <= 0
  
  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]
    
    return loss

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.log('training_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"train_loss": loss}
    return {"loss": loss, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    self.log('avg_training_loss', avg_train_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"avg_train_loss": avg_train_loss}
    return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"val_loss": loss}
    return {"val_loss": loss, "log": tensorboard_logs}
  
  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    print(avg_loss)
    tensorboard_logs = {"val_loss": avg_loss}
    self.log('avg_val_loss', avg_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self,
                     epoch=None, 
                     batch_idx=None, 
                     optimizer=None, 
                     optimizer_idx=None, 
                     optimizer_closure=None, 
                     on_tpu=None, 
                     using_native_amp=None, 
                     using_lbfgs=None
                     ):

    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [ ]:
# The below code is adapted from:
# https://github.com/huggingface/transformers/blob/master/examples/multiple-choice/utils_multiple_choice.py

@dataclass(frozen=True)
class InputExample:
    """
    A single training/test example for multiple choice
    Args:
        example_id: Unique id for the example.
        question: string. The untokenized text of the second sequence (question).
        contexts: list of str. The untokenized text of the first sequence (context of corresponding question).
        endings: list of str. multiple choice's options. Its length must be equal to contexts' length.
        label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """

    name: str
    ingredients: str
    steps: List[str]
    label: Optional[str]

class Split(Enum):
    train = "train"
    dev = "dev"
    test = "test"

class DataProcessor:
    """Base class for data converters for multiple choice data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

class RecipeProcessor(DataProcessor):
    """Processor for the SWAG data set."""

    def get_train_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} train".format(data_dir))
        return self._create_examples(self._read_csv(os.path.join(data_dir, "train.csv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} dev".format(data_dir))
        return self._create_examples(self._read_csv(os.path.join(data_dir, "val.csv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} dev".format(data_dir))
        raise ValueError(
            "For swag testing, the input file does not contain a label column. It can not be tested in current code"
            "setting!"
        )
        return self._create_examples(self._read_csv(os.path.join(data_dir, "test.csv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1", "2", "3"]

    def _read_csv(self, input_file):
        with open(input_file, "r", encoding="utf-8") as f:
            return list(csv.reader(f))

    def _create_examples(self, lines: List[List[str]], type: str):
        """Creates examples for the training and dev sets."""

        examples = [
            InputExample(
                name=line[1],
                # common beginning of each
                # choice is stored in "sent2".
                ingredients=eval(line[3]),
                steps=eval(line[2]),
                label=eval(line[2]),
            )
            for line in lines[1:]  # we skip the line with the column names
        ]

        return examples

In [ ]:
class RecipeDataset(Dataset):
    
    def __init__(self, tokenizer, data_dir, type_path,  max_len=512, mask_percent=0.4, mode='fill'):
        self.data_dir = data_dir
        self.type_path = type_path
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.ingredients = []
        self.names = []
        self.mode = mode
        self.mask_percent = mask_percent
        
        self.proc = RecipeProcessor()
        self._build()
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, 
                "target_ids": target_ids, "target_mask": target_mask}
  
    def __len__(self):
        return len(self.inputs)
  
    def _build(self):
        if self.type_path == 'train':
            examples = self.proc.get_train_examples(self.data_dir)
        else:
            examples = self.proc.get_dev_examples(self.data_dir)
    
        for i, example in enumerate(examples):
            if i % 10000 == 0:
                print(i)
            self._create_features(example)
    
    def _create_shuffle_completion(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          input_words = []
          label = []
        else:
          
          mask_words = np.round(0.5*total_words).astype(int)
          if np.random.uniform() < 0.5:
            input_words = words[:mask_words] + ['<extra_id_0>']
            label = ['<extra_id_0>'] + words[mask_words:] + ['<extra_id_1>']
          else:
            input_words = ['<extra_id_0>'] + words[mask_words:]
            label = ['<extra_id_0>'] + words[:mask_words] + ['<extra_id_1>']

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
    
    def _create_completion(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          input_words = []
          label = []
        else:
          mask_words = np.round(0.5*total_words).astype(int)
          input_words = words[:mask_words] + ['<extra_id_0>']
          label = ['<extra_id_0>'] + words[mask_words:] + ['<extra_id_1>']

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
  
    def _create_masks(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          mask_words = 0
        else:
          mask_words = np.round(self.mask_percent*total_words).astype(int)

        mask_indices = np.random.choice(np.arange(total_words), mask_words)
        
        input_words = []
        label = []
        j = 0
        
        for i, word in enumerate(words):
            if i in mask_indices:
                # Add a sentinel token in place of the word to be masked
                input_words.append('<extra_id_{}>'.format(j))
                # Add this word's token to the label list
                label.append('<extra_id_{}>'.format(j))
                # Add the corresponding label to the list
                label.append(word)
                # Step the counter by 1
                j+=1
            # If this index is not among the sampled indices 
            # just append the word
            else:
                input_words.append(word)
         
        # Add in an extra ID token like the format says
        label.append('<extra_id_{}>'.format(j))

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
    
    def _create_features(self, example):
    
        ingredients_ = ",".join(example.ingredients)
        name_ = example.name
        
        if self.mode == 'fill':
          masked_, label_ = self._create_masks(example.steps)
        
        elif self.mode == 'complete':
          masked_, label_ = self._create_completion(example.steps)

        elif self.mode == 'shuffle_complete':
          masked_, label_ = self._create_completion(example.steps)  
                                        
        input_ = "name: %s  ingredients: %s masked: %s </s>" % (name_, ingredients_, masked_)
        target =  label_ + " </s>"
        
        # tokenize inputs
        tokenized_inputs = self.tokenizer.batch_encode_plus(
            [input_], max_length = 200, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )
        # tokenize targets
        tokenized_targets = self.tokenizer.batch_encode_plus(
            [target], max_length = 150, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )

        self.inputs.append(tokenized_inputs)
        self.targets.append(tokenized_targets)
        self.ingredients.append(example.ingredients)
        self.names.append(example.name)

In [ ]:
def generate_loop(checkpoint_dir='drive/MyDrive/recipe-generation/t5_fill_blanks/', 
                  checkpoint_name = 'exp_7_fb_ef.ckpt', 
                  experiment_name = 'experiment_7',
                  mode = 'fill'):
  '''
  ---------
  Generation loop
  --------
  '''
  checkpoint_path = os.path.join(checkpoint_dir, checkpoint_name)
  model = T5FineTuner.load_from_checkpoint(checkpoint_path)
  tokenizer = T5Tokenizer.from_pretrained("t5-base")

  dataset =  RecipeDataset(tokenizer, data_dir='drive/MyDrive/recipe-generation/data', type_path='val', mode=mode)
  loader = DataLoader(dataset, batch_size=32, num_workers=2)

  model.model.eval()
  outputs = []
  targets = []

  for i, batch in enumerate(tqdm(loader)):

    # Need to change max. length argument
    outs = model.model.generate(input_ids=batch['source_ids'], 
                                attention_mask=batch['source_mask'], 
                                max_length=150)
    dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
    target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

    if i%100 == 0:
      print(dec)
      print(target)

    outputs.extend(dec)
    targets.extend(target)

  results = [dataset.names, dataset.ingredients, outputs, targets]
  results_path = os.path.join(checkpoint_dir, '{}_outputs.pkl'.format(experiment_name))
  
  return(results) 

In [ ]:
def write_results(results, results_path):
  '''
  Pickle and write results 
  to file
  '''
  with open(results_path, 'wb') as f:
    pickle.dump(results, f)
    f.close()

In [ ]:
def read_results(results_path):
    '''
    ----------
    Load results from pkl. file
    -----------
    '''
    # Open statement within context
    with open(results_path, 'rb') as f:
      # Load statement
      results = pickle.load(f)
    # Return statement
    return(results)

In [40]:
checkpoint_dir='drive/MyDrive/recipe-generation/t5_fill_blanks/'

In [41]:
checkpoint_name = 'exp_3_fb.ckpt' 
experiment_name = 'experiment_8'
mode = 'fill'

results = generate_loop(checkpoint_name=checkpoint_name, experiment_name=experiment_name, mode=mode)

0


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


10000



  0%|          | 1/568 [00:07<1:08:36,  7.26s/it]

['a, the and and stirring occasionally, of the are tender, mixture bowl let stand stir in the and at room', 'sift flour into a bowl yeast and the ingredients a batter place in a of egg into cook over medium appear pan to remove from remaining or favorite of', 'add remaining hour tender if soup soup 1 2 to the soup to', ', blend flour,, and for stir pie plate minutes out clean inserted in', 'a saucepan heat garlic saute onions translucent add for, vegetable broth, and ground black make covered with necessary little water cover and simmer a the rest pepper taste', 'of 9x13 pan add the, coconut do not allow each its condensed completely bake 20 cool cutting size bars from', 'in large, raisins in hot blend 5 combine sugar cooked until smooth, stirring constantly thickened oven grease a dish set rinse vanilla baking to 45 minutes or thin 10 minutes rose sprinkled or the raisins 4 before', 'oven line baking spray muffins dry in large mix well combine margarine nuts using powder, spice, soda 

...


 71%|███████   | 401/568 [41:33<16:33,  5.95s/it]

['350f the butter, vanilla and water in large a separate together baking ingredients the creamed stir in shape place 2-inches an bake for 10-12 until around careful over them', '350f 13x9-inch large, butter and salt with medium until crumbs press mixture the bake corn chips, water,, stirring the mixture stir the cashews coated hot minutes cool completely into', 'thick in seeds more 1 / seeds from to sprouts add a bowl', '1 the tartar royal, just teaspoon use as a powder baking mixture in a', 'preheat degrees peanut and well in salt well stir in m&ms chips bake 10-12 soft cookies, minutes soft remove to', 'tablespoon sugar butter and stir eggs flour and salt to and smooth and transfer to, cover and rise place in microwave 8-inch square with a spatula, punch, and out 9-inch cut buns crosswise a greased stand hour with pine nuts', "preheat 350 mix onions a x x dish bake top light, bake until golden watch carefully carefully don't", 'grill on it from the fish with vegetable season the fish


 88%|████████▊ | 501/568 [51:41<06:54,  6.18s/it]

['oil a large frying cook haloumi,, side remove haloumi mixture plate stirring 3 are through add cook until melted stir in the lemon bring arrange capers platter with oil and sprinkle immediately', 'brown beef with sauce,, garlic and tomatoes add sugar cumin salt and or for to serve in with top with i garnish and / this recipe for', 'and large together mayonnaise and green powder macaroni, carrot, toss to well least 2 blend', ', and orange juice can distributed add, milk until', 'a small bowl juice of the, and place bowl to the steaks cover and refrigerate for hours, marinade discard for 10 each side, desired', 'to 350f baking with spray and of soda salt beat until pour prepared and bake about 25', 'the bowl add mix well high', 'in of brownie ingredients order following order for a layer is the layer a / piece of ribbon and the in a of water, olive, and vanilla well greased pan and in 350 oven for minutes or until a toothpick comes clean', 'the of coconut on prepared in, combine sugar 


100%|██████████| 568/568 [58:24<00:00,  6.17s/it]


'a, the and and stirring occasionally, of the are tender, mixture bowl let stand stir in the and at room'

'a, the and and stirring occasionally, of the have burst, sauce bowl let stand fold in the and at room'

'drive/MyDrive/recipe-generation/t5_fill_blanks/experiment_7_outputs.pkl'